In [3]:
from huggingface_hub import hf_hub_download 

hf_hub_download(repo_id="ai-lab/MBD-mini", filename="ptls.tar.gz", repo_type="dataset", local_dir="/kaggle/working/")
hf_hub_download(repo_id="ai-lab/MBD-mini", filename="targets.tar.gz", repo_type="dataset", local_dir="/kaggle/working/")

ptls.tar.gz:   0%|          | 0.00/1.40G [00:00<?, ?B/s]

targets.tar.gz:   0%|          | 0.00/7.93M [00:00<?, ?B/s]

'/kaggle/working/targets.tar.gz'

In [1]:
!pip install pyspark
!pip install pytorch-lifestream

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.3 MB/s eta 0:00:000:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.4-py2.py3-none-any.whl size=317849765 sha256=835b3927a7fda6a2ec615c5773b9494c5cfefd4ef78cb97538b73763a99a397a
  Stored in directory: /root/.cache/pip/wheels/d9/1c/98/31e395a42d1735d18d42124971ecbbade844b50bb9845b6f4a
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.3 MB/s eta 0:00:00
  Created wheel for pytorch-lifestream: filename=pytorch_lifestream-0.6.0-py3-none-any.whl size=274640 sha256=b94251ff2b7a6a5e1f3f50fa1240cbd3e3ad34f320dafae66a39c1b6bbd930fe
  Stored in directory: /root/.cache/pip/wheels/90/76/b4/0a944bc7c5a69201e4d757cc54886971117a2a58174

In [4]:
!tar -xf ptls.tar.gz
!tar -xf targets.tar.gz

In [28]:
import pandas as pd
import numpy as np
import os

import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T
import time
import datetime
from ptls.data_load.datasets import ParquetDataset, ParquetFiles
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType
from tqdm.notebook import tqdm
from ptls.preprocessing import PysparkDataPreprocessor
import pytorch_lightning as pl
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter, FeatureFilter
from ptls.data_load.iterable_processing.iterable_seq_len_limit import ISeqLenLimit
from ptls.data_load.iterable_processing.to_torch_tensor import ToTorch
from ptls.frames.coles import CoLESModule
from ptls.frames import PtlsDataModule
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices

In [22]:
spark_conf = pyspark.SparkConf()
spark_conf.setMaster("local[*]").setAppName("JoinModality")
spark_conf.set("spark.driver.maxResultSize", "16g")
spark_conf.set("spark.executor.memory", "32g")
spark_conf.set("spark.executor.memoryOverhead", "16g")
spark_conf.set("spark.driver.memory", "32g")
spark_conf.set("spark.driver.memoryOverhead", "16g")
spark_conf.set("spark.cores.max", "24")
spark_conf.set("spark.sql.shuffle.partitions", "200")
spark_conf.set("spark.local.dir", "../../spark_local_dir")


spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark.sparkContext.getConf().getAll()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


[('spark.driver.port', '46849'),
 ('spark.app.id', 'local-1737081552627'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.executor.memoryOver

In [23]:
spark

# With MM Dataset

In [24]:
!mkdir /kaggle/working/mm_dataset

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [31]:
TRX_DATA_PATH = '/kaggle/working/ptls/trx/'
GEO_DATA_PATH = '/kaggle/working/ptls/geo/'
DIAL_DATA_PATH = '/kaggle/working/ptls/dialog/'

MM_DATA_PATH = '/kaggle/working/mm_dataset'
MMT_DATA_PATH = '/kaggle/working/mm_dataset_supervised'

TARGETS_DATA_PATH = '/kaggle/working/targets/'

In [26]:
def rename_col(df, prefix, col_id='client_id'):
    new_column_names = [f"{prefix}_{col}" for col in df.columns if col != col_id]
    old_column_names = [col for col in df.columns if col != col_id]
    for old_col, new_col in zip(old_column_names, new_column_names):
        df = df.withColumnRenamed(old_col, new_col)
    return df

In [29]:
class AddNulls:
    def __init__(self, max_date, dict_sizes, col_time, col_id, col_target=None, is_null_start=True):
        
        self.max_date = int(max_date // 86400)
        self.dict_sizes = dict_sizes
        self.col_time = col_time
        self.col_id = col_id
        # self.col_target = col_target
        self.is_null_start = is_null_start

    def step(self, features):
        
        event_time_int = np.array(features[self.col_time], dtype=int).tolist()  # Преобразуем значения в целые числа
        filled_event_time = []
        event_time_int = [int(event_time_int[i] // 86400) for i in range(len(event_time_int))]
        prev_time = event_time_int[0]

        # Заполнение пропусков в event_time
        for time in event_time_int:
            time = time
            if time > prev_time + 1:
                # Если между предыдущим временем и текущим есть пропуск, заполняем пропуски
                for t in range(prev_time + 1, time):
                    filled_event_time.append(t)
            filled_event_time.append(time)
            prev_time = time
        for time in range(event_time_int[-1], self.max_date):
            filled_event_time.append(time)

        # Обработка остальных признаков
        filled_features = {key: [] for key in self.dict_sizes.keys()}

        event_time_index = 0  # Индекс для event_time

        # Пробегаем по всем значениям в filled_event_time
        for i in range(len(filled_event_time)):
            # Если текущий момент времени совпадает с событием в исходном event_time
            if event_time_index < len(event_time_int) and filled_event_time[i] == event_time_int[event_time_index]:
                # Для каждого признака сохраняем значение, если оно есть в event_time
                for key in self.dict_sizes.keys():
                    filled_features[key].append(features[key][event_time_index])
                event_time_index += 1
            else:
                # Если нет события, заполняем значением из dict_sizes для каждого признака
                for key in self.dict_sizes.keys():
                    filled_features[key].append(self.dict_sizes[key])

        filled_features[self.col_time] = filled_event_time
        filled_features[self.col_id] = features[self.col_id]
        # filled_features[self.col_target] = features[self.col_target]
        return filled_features

trx_dict_sizes = {
    'trx_amount': 0.0,
    'trx_event_type': 0,
    'trx_event_subtype': 0,
    'trx_currency': 0,
    'trx_src_type11': 0,
    'trx_src_type12': 0,
    'trx_dst_type11': 0,
    'trx_dst_type12': 0,
    'trx_src_type21': 0,
    'trx_src_type22': 0,
    'trx_src_type31': 0,
    'trx_src_type32': 0
}


s = "31/12/2022"
max_date = time.mktime(datetime.datetime.strptime(s, "%d/%m/%Y").timetuple())
trx_col_time = "trx_event_time"
col_id = "client_id"

trx_processor = AddNulls(max_date, trx_dict_sizes, trx_col_time, col_id)

dial_dict_sizes = {
    'dial_embedding': [[0 for i in range(786)]]
}
dial_col_time = "dial_event_time"
dial_processor = AddNulls(max_date, dial_dict_sizes, dial_col_time, col_id)

**Create multimodal dataset with ADDNULLS**

In [34]:
from pyspark.sql.functions import array

targets = spark.read.parquet(os.path.join(TARGETS_DATA_PATH , f'fold={0}'))
targets = targets.withColumn(
    "targets",
    array(targets.target_1, targets.target_2, targets.target_3, targets.target_4)
)
targets.show()

+--------------------+----------+--------+--------+--------+--------+-----------+---------------+------------+
|           client_id|       mon|target_1|target_2|target_3|target_4|trans_count|diff_trans_date|     targets|
+--------------------+----------+--------+--------+--------+--------+-----------+---------------+------------+
|0011eb856025cc736...|2022-02-28|       0|       0|       0|       0|         32|              0|[0, 0, 0, 0]|
|0011eb856025cc736...|2022-03-31|       0|       0|       0|       0|         69|              0|[0, 0, 0, 0]|
|0011eb856025cc736...|2022-04-30|       0|       0|       0|       0|         97|              0|[0, 0, 0, 0]|
|0011eb856025cc736...|2022-05-31|       0|       0|       0|       0|        125|              1|[0, 0, 0, 0]|
|0011eb856025cc736...|2022-06-30|       0|       0|       0|       0|        156|              0|[0, 0, 0, 0]|
|0011eb856025cc736...|2022-07-31|       0|       0|       0|       0|        192|              0|[0, 0, 0, 0]|
|

In [41]:
preprocessor_target = PysparkDataPreprocessor(
    col_id="client_id",
    col_event_time="mon",
    event_time_transformation="dt_to_timestamp",
    cols_identity=["target_1", "target_2", "target_3", "target_4"],
)


def create_infer_schema(data):
    first_row = data[0]
    fields = []

    for key, value in first_row.items():
        if isinstance(value, list):
            # Если значение — список, то это будет ArrayType
            if isinstance(value[0], int):
                fields.append(StructField(key, ArrayType(IntegerType()), True))
            elif isinstance(value[0], float):
                fields.append(StructField(key, ArrayType(FloatType()), True))
            elif isinstance(value[0], str):
                fields.append(StructField(key, ArrayType(StringType()), True))
            else:
                fields.append(StructField(key, ArrayType(StringType()), True))  # Default to StringType
        else:
            # Если значение не список, то это обычный тип
            if isinstance(value, int):
                fields.append(StructField(key, IntegerType(), True))
            elif isinstance(value, float):
                fields.append(StructField(key, FloatType(), True))
            elif isinstance(value, str):
                fields.append(StructField(key, StringType(), True))
            else:
                fields.append(StructField(key, StringType(), True))  # Default to StringType

    schema = StructType(fields)
    return schema

for fold in tqdm(range(0, 5)):
    trx = spark.read.parquet(os.path.join(TRX_DATA_PATH, f'fold={fold}'))
    # geo = spark.read.parquet(os.path.join(GEO_DATA_PATH, f'fold={fold}'))
    dial = spark.read.parquet(os.path.join(DIAL_DATA_PATH, f'fold={fold}'))
    trx = rename_col(trx, 'trx')
    # trx = trx.rdd.map(lambda x: trx_processor.step(x.asDict()))
    # schema = create_infer_schema([trx.first()])
    # trx = spark.createDataFrame(trx, schema=schema)
    # geo = rename_col(geo, 'geo')
    dial = rename_col(dial, 'dial')
    # dial = dial.rdd.map(lambda x: dial_processor.step(x.asDict()))
    # schema = create_infer_schema([dial.first()])
    # dial = spark.createDataFrame(dial, schema=schema)
    
    mm_dataset = trx.join(dial, on='client_id', how='outer')
    
    targets = spark.read.parquet(os.path.join(TARGETS_DATA_PATH , f'fold={fold}'))
    targets = targets.withColumn(
    "targets",
    array(targets.target_1, targets.target_2, targets.target_3, targets.target_4)
)
    targets = preprocessor_target.fit_transform(targets)

    mm_dataset = mm_dataset.join(targets, on='client_id', how='left').drop(*[
        'target_1',
        'target_2',
        'target_3',
        'target_4',
        'trans_count',
        'diff_trans_date'
    ])
    mm_dataset.write.mode('overwrite').parquet(os.path.join(MMT_DATA_PATH, f'fold={fold}'))
    del trx
    del dial
    del mm_dataset

  0%|          | 0/5 [00:00<?, ?it/s]

In [63]:
import torch
import numpy as np
import pandas as pd
import calendar
from glob import glob
from ptls.data_load.utils import collate_feature_dict

from ptls.data_load.iterable_processing_dataset import IterableProcessingDataset
from datetime import datetime
from ptls.data_load.padded_batch import PaddedBatch

class TargetToTorch(IterableProcessingDataset):
    def __init__(self, col_target):
        super().__init__()
        self.col_target = col_target

    def __iter__(self):
        for rec in self._src:
            features = rec[0] if type(rec) is tuple else rec
            features[self.col_target] = np.stack(np.array(features[self.col_target]))
            features[self.col_target] = torch.tensor(features[self.col_target])
            yield features


class GetSplit(IterableProcessingDataset):
    def __init__(
        self,
        start_month,
        end_month,
        year=2022,
        col_id='client_id',
        col_time='event_time'
    ):
        super().__init__()
        self.start_month = start_month
        self.end_month = end_month
        self._year = year
        self._col_id = col_id
        self._col_time = col_time
        
    def __iter__(self):
        for rec in self._src:
            for month in range(self.start_month, self.end_month+1):
                features = rec[0] if type(rec) is tuple else rec
                features = features.copy()
                
                if month == 12:
                    month_event_time = datetime(self._year + 1, 1, 1).timestamp()
                else:
                    month_event_time = datetime(self._year, month + 1, 1).timestamp()
                    
                year_event_time = datetime(self._year, 1, 1).timestamp()
                
                mask = features[self._col_time] < month_event_time
                
                for key, tensor in features.items():
                    if key.startswith('target'):
                        features[key] = tensor[month - 1].tolist()    
                    elif key != self._col_id:
                        features[key] = tensor[mask] 
                            
                features[self._col_id] += '_month=' + str(month)

                yield features

def collate_feature_dict_with_target(batch, col_id='client_id', target_col_names=None):
    batch_ids = []
    target_cols = []
    for sample in batch:
        batch_ids.append(sample[col_id])
        del sample[col_id]
        
        if target_col_names is not None:
            for target_col in target_col_names:
                target_cols.append(sample[target_col])
                del sample[target_col]
            
    padded_batch = collate_feature_dict(batch)
    if target_col_names is not None:
        return padded_batch, batch_ids, target_cols
    return padded_batch, batch_ids

In [102]:
from ptls.data_load.datasets import ParquetDataset
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.data_load.iterable_processing.iterable_seq_len_limit import ISeqLenLimit
from ptls.data_load.iterable_processing.to_torch_tensor import ToTorch
train = ParquetDataset(
    data_files=[
        os.path.join(MMT_DATA_PATH, f'fold={0}'),
        os.path.join(MMT_DATA_PATH, f'fold={1}'),
        os.path.join(MMT_DATA_PATH, f'fold={2}')
    ],
    i_filters=[
        SeqLenFilter(min_seq_len=32),
        ISeqLenLimit(max_seq_len=4096),
        ToTorch(),
        TargetToTorch(col_target='targets'),
    ],
    shuffle_files=True
)
valid = ParquetDataset(
    data_files=[
        os.path.join(MMT_DATA_PATH, f'fold={3}')
    ],
    i_filters=[
        SeqLenFilter(min_seq_len=32),
        ISeqLenLimit(max_seq_len=4096),
        ToTorch(),
        TargetToTorch(col_target='targets'),
    ],
    shuffle_files=True
)

In [122]:
next(iter(train))

{'client_id': '000e047a31e50ba35f71c81962b4eb0b9a2d6080cf23a14f91447317dd14c788',
 'trx_event_time': tensor([1609490380, 1610003909, 1610180496, 1610183393, 1610264466, 1610266194,
         1610442309, 1610519884, 1610612313, 1610687058, 1610867659, 1611209220,
         1611379348, 1611389622, 1611553654, 1611737692, 1611991352, 1612002459,
         1612174968, 1612250928, 1612253101, 1612419223, 1612421720, 1612434125,
         1612504321, 1612589076, 1612673828, 1612773491, 1612868722, 1613018794,
         1613026708, 1613104541, 1613113892, 1613118370, 1613289751, 1613297280,
         1613378254, 1613638876, 1613799989, 1613809924, 1613811768, 1613887740,
         1613901739, 1613978649, 1614057274, 1614495629, 1614495765, 1614581690,
         1614669285, 1614924793, 1615112066, 1615184054, 1615447812, 1615707336,
         1616228131, 1616406936, 1616823175, 1617090479, 1617182075, 1617608630,
         1617695825, 1617698777, 1617784066, 1617880952, 1617962327, 1618044585,
         

**Model**

In [138]:
from ptls.data_load.feature_dict import FeatureDict
from collections import defaultdict
class MultiModalDiffSplitDataset(FeatureDict, torch.utils.data.Dataset):
    def __init__(
        self,
        data,
        splitters,
        source_features,
        col_id,
        source_names,
        col_time='event_time',
        *args, **kwargs
    ):
        """
        Dataset for multimodal learning.
        Parameters:
        -----------
        data:
            concatinated data with feature dicts.
        splitter:
            object from from `ptls.frames.coles.split_strategy`.
            Used to split original sequence into subsequences which are samples from one client.
        source_features:
            list of column names 
        col_id:
            column name with user_id
        source_names:
            column name with name sources
        col_time:
            column name with event_time
        """
        super().__init__(*args, **kwargs)
        
        self.data = data
        self.splitters = splitters
        self.col_time = col_time
        self.col_id = col_id
        self.source_names = source_names
        self.source_features = source_features
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        feature_arrays = self.data[idx]
        split_data = self.split_source(feature_arrays)
        return self.get_splits(split_data)
    
    def __iter__(self):
        for feature_arrays in self.data:
            split_data = self.split_source(feature_arrays)
            yield self.get_splits(split_data)
            
    def split_source(self, feature_arrays):
        res = defaultdict(dict)
        for feature_name, feature_array in feature_arrays.items():
            if feature_name == self.col_id:
                res[self.col_id] = feature_array
            else:
                source_name, feature_name_transform = self.get_names(feature_name)
                res[source_name][feature_name_transform] = feature_array
        for source in self.source_names:
            if source not in res:
                res[source] = {source_feature: torch.tensor([]) for source_feature in self.source_features[source]}
        return res
    
    def get_names(self, feature_name):
        idx_del = feature_name.find('_')
        return feature_name[:idx_del], feature_name[idx_del + 1:]
    
    def get_splits(self, feature_arrays):
        res = {}
        print(feature_arrays)
        for source_name, feature_array in feature_arrays.items():
            if source_name != self.col_id:
                local_date = feature_array[self.col_time]
                if source_name not in self.splitters:
                    continue
                indexes = self.splitters[source_name].split(local_date)
                res[source_name] = [{k: v[ix] for k, v in feature_array.items() if self.is_seq_feature(k, v)} for ix in indexes]
        return res
        
    def collate_fn(self, batch, return_dct_labels=False):
        dict_class_labels = get_dict_class_labels(batch)
        batch = reduce(lambda x, y: {k: x[k] + y[k] for k in x if k in y}, batch)
        padded_batch = collate_multimodal_feature_dict(batch)
        if return_dct_labels:
            return padded_batch, dict_class_labels
        return padded_batch, dict_class_labels[list(dict_class_labels.keys())[0]]

class MultiModalDiffSplitIterableDataset(MultiModalDiffSplitDataset, torch.utils.data.IterableDataset):
    pass

In [139]:
from ptls.frames import PtlsDataModule
from ptls.frames.coles.split_strategy import SampleSlices
trainer = pl.Trainer(
    max_epochs=12,
    accelerator="cuda" if torch.cuda.is_available() else "cpu",
    enable_progress_bar=True,
    gradient_clip_val=0.5,
    log_every_n_steps=50,
    limit_val_batches=512
)

data_module = PtlsDataModule(
    train_data=MultiModalDiffSplitIterableDataset(
        data=train,
        splitters={
            "trx": SampleSlices(
                split_count=2,
                cnt_min=32,
                cnt_max=180
            ),
            "dial": SampleSlices(
                split_count=2,
                cnt_min=2,
                cnt_max=10
            ),
        },
        source_features={
            "trx": [
                "trx_event_type",
                "trx_event_subtype",
                "trx_src_type11",
                "trx_src_type12",
                "trx_dst_type11",
                "trx_dst_type12",
                "trx_src_type22",
                "trx_src_type32",
                "trx_event_time"
            ],
            "dial": [
                "dial_embedding",
                "dial_event_time"
            ],
        },
        col_id='client_id',
        col_time='event_time',
        source_names=['trx', 'dial'],
    )
)

In [140]:
from ptls.frames.abs_module import ABSModule


def first(iterable, default=None):
    iterator = iter(iterable)
    return next(iterator, default)


class M3CoLESModule(ABSModule):
    """
    Multi-Modal Matching
    Contrastive Learning for Event Sequences ([CoLES](https://arxiv.org/abs/2002.08232))

    Subsequences are sampled from original sequence.
    Samples from the same sequence are `positive` examples
    Samples from the different sequences are `negative` examples
    Embeddings for all samples are calculated.
    Paired distances between all embeddings are calculated.
    The loss function tends to make positive distances smaller and negative ones larger.

    Parameters
        seq_encoder:
            Model which calculate embeddings for original raw transaction sequences
            `seq_encoder` is trained by `CoLESModule` to get better representations of input sequences
        head:
            Model which helps to train. Not used during inference
            Can be normalisation layer which make embedding l2 length equals 1
            Can be MLP as `projection head` like in SymCLR framework.
        loss:
            loss object from `ptls.frames.coles.losses`.
            There are paired and triplet loss. They are required sampling strategy
            from `ptls.frames.coles.sampling_strategies`. Sampling strategy takes a relevant pairs or triplets from
            pairwise distance matrix.
        validation_metric:
            Keep None. `ptls.frames.coles.metric.BatchRecallTopK` used by default.
        optimizer_partial:
            optimizer init partial. Network parameters are missed.
        lr_scheduler_partial:
            scheduler init partial. Optimizer are missed.

    """
    def __init__(self,
                 seq_encoders=None,
                 mod_names=None,
                 head=None,
                 loss=None,
                 validation_metric=None,
                 optimizer_partial=None,
                 lr_scheduler_partial=None):
        torch.set_float32_matmul_precision('high')
        if head is None:
            head = Head(use_norm_encoder=True)

        if loss is None:
            loss = ContrastiveLoss(margin=0.5,
                                   sampling_strategy=HardNegativePairSelector(neg_count=5))

        if validation_metric is None:
            validation_metric = BatchRecallTopK(K=4, metric='cosine')
        
        for k in seq_encoders.keys():
            if type(seq_encoders[k]) is str:
                seq_encoders[k] = seq_encoders[seq_encoders[k]]
                
        super().__init__(validation_metric,
                         first(seq_encoders.values()),
                         loss,
                         optimizer_partial,
                         lr_scheduler_partial)
        
        self.seq_encoders = torch.nn.ModuleDict(seq_encoders)
        self._head = head   
        self.y_h_cache = {'train':[], 'valid': []}
        
    @property
    def metric_name(self):
        return 'recall_top_k'

    @property
    def is_requires_reduced_sequence(self):
        return True
    
    def forward(self, x):
        res = {}
        
        for mod_name in x.keys():
            res[mod_name] = self.seq_encoders[mod_name](x[mod_name])
            
        return res

    def shared_step(self, x, y):
        y_h = self(x)
        
        if self._head is not None:
            y_h_head = {k: self._head(y_h_k) for k, y_h_k in y_h.items()}
            y_h = y_h_head
            
        return y_h, y
    
    def _one_step(self, batch, _, stage):
        y_h, y = self.shared_step(*batch)
        y_h_list = list(y_h.values())
        loss = self._loss(torch.cat(y_h_list), torch.cat([y, y]))
        self.log(f'loss/{stage}', loss.detach())
        
        x, y = batch
        for mod_name, mod_x in x.items():
            self.log(f'seq_len/{stage}/{mod_name}', x[mod_name].seq_lens.float().mean().detach(), prog_bar=True)
        
        if stage == "valid":
            n, d = y_h_list[0].shape
            y_h_concat = torch.zeros((2*n, d), device = y_h_list[0].device)
            
            for i in range(2):
                y_h_concat[range(i,2*n,2)] = y_h_list[i] 
            
            if len(self.y_h_cache[stage]) <= 380:
                self.y_h_cache[stage].append((y_h_concat.cpu(), {k: y_h_k.cpu() for k, y_h_k in y_h.items()} , 
                                             {k:x_k.seq_lens.cpu() for k, x_k in x.items()})) 
    
        return loss
    
    def training_step(self, batch, _):
        return self._one_step(batch, _, "train")
    
    def validation_step(self, batch, _):
        return self._one_step(batch, _, "valid")
    
    def on_validation_epoch_end(self):        
        #len_intervals = [(0, 10), (10, 20), (20, 30), (30, 40), (40, 60), (60, 80), (80, 120), (120, 160), (160, 240)]
        self.log_recall_top_K(self.y_h_cache['valid'], len_intervals=None, stage="valid", K=100)
        self.log_recall_top_K(self.y_h_cache['valid'], len_intervals=None, stage="valid", K=50)
        self.log_recall_top_K(self.y_h_cache['valid'], len_intervals=None, stage="valid", K=1)
        
        
        del self.y_h_cache["valid"]
        self.y_h_cache["valid"] = []
        
    def log_recall_top_K(self, y_h_cache, len_intervals=None, stage="valid", K=100):
        y_h = torch.cat([item[0] for item in y_h_cache], dim = 0)
        y_h_mods = defaultdict(list)
        seq_lens_dict = defaultdict(list)
        
        for item in y_h_cache:
            for k, emb in item[1].items():
                y_h_mods[k].append(emb)
                
            for k, l in item[2].items():
                seq_lens_dict[k].append(l)
        
        y_h_mods = {k: torch.cat(el, dim=0) for k ,el in y_h_mods.items()}
        seq_lens_dict = {k: torch.cat(el) for k ,el in seq_lens_dict.items()}

        #n, _ = y_h.shape
        #y = torch.zeros((n,)).cpu().long()
        #y[range(0,n,2)] = torch.arange(0, n//2)
        #y[range(1,n,2)] = torch.arange(0, n//2)
        #computed_metric = metric_real_recall_top_K(y_h, y, K=100)
        y_h_bank, y_h_rmb = list(y_h_mods.values())
        computed_metric_b2r = metric_recall_top_K_for_embs(y_h_bank, y_h_rmb, torch.arange(y_h_rmb.shape[0]), K=K)
        computed_metric_r2b = metric_recall_top_K_for_embs(y_h_rmb, y_h_bank, torch.arange(y_h_rmb.shape[0]), K=K)
        
        if len_intervals != None:
            for mod, seq_lens in seq_lens_dict.items():
                for start, end in len_intervals:
                    mask = ((seq_lens > start) & (seq_lens <= end))

                    if torch.any(mask):
                        #y_h_filtered = y_h[mask.repeat_interleave(2)]
                        y_h_bank_filtered = y_h_bank[mask]
                        y_h_rmb_filtered = y_h_rmb[mask]

                        #y = torch.div(torch.arange(len(y_h_filtered)), 2, rounding_mode='floor')
                        #recall = metric_real_recall_top_K(y_h_filtered, y, K=100)
                        recall_r2b = metric_recall_top_K_for_embs(y_h_rmb_filtered, y_h_bank_filtered, torch.arange(y_h_rmb_filtered.shape[0]), K=100)
                        recall_b2r = metric_recall_top_K_for_embs(y_h_bank_filtered, y_h_rmb_filtered, torch.arange(y_h_rmb_filtered.shape[0]), K=100)

                        #self.log(f"{mode}/R@100_len_from_{start}_to_{end}", recall, prog_bar=True)
                        self.log(f"{stage}/{mod}/r2b_R@100_len_from_{start}_to_{end}", recall_r2b, prog_bar=True)
                        self.log(f"{stage}/{mod}/b2r_R@100_len_from_{start}_to_{end}", recall_b2r, prog_bar=True)
        
        #self.log(f"{mode}/R@100", computed_metric, prog_bar=True)
        self.log(f"{stage}/click2trx_R@{K}", computed_metric_r2b, prog_bar=True)
        self.log(f"{stage}/trx2click_R@{K}", computed_metric_b2r, prog_bar=True)

In [141]:
from ptls.frames.coles.metric import BatchRecallTopK
from ptls.nn.trx_encoder.encoders import IdentityEncoder
import ptls
from functools import partial
head = ptls.nn.Head(
    input_size=128,
    use_norm_encoder=True,
    hidden_layers_sizes=[128, 128],
    objective="regression",
    num_classes=128
)
seq_encoders = {
    "trx": ptls.nn.RnnSeqEncoder(
        trx_encoder=ptls.nn.TrxEncoder(
            norm_embeddings=False,
            embeddings_noise=0.003,
            linear_projection_size=32,
            embeddings={
                'event_type': {"in": 58, "out": 24},
                'event_subtype': {"in": 59, "out": 24},
                'src_type11': {"in": 85, "out": 24},
                'src_type12': {"in": 349, "out": 24},
                'dst_type11': {"in": 84, "out": 24},
                'dst_type12': {"in": 417, "out": 24},
                'src_type22': {"in": 90, "out": 24},
                'src_type32': {"in": 91, "out": 24}
            }
        ),
        type="gru",
        hidden_size=128
    ),
    "dial": ptls.nn.RnnSeqEncoder(
        trx_encoder=ptls.nn.TrxEncoder(
            embeddings_noise=0.003,
            linear_projection_size=32,
            custom_embeddings={
                "embedding": IdentityEncoder(output_size=768)
            }
        ),
        type="gru",
        hidden_size=128
    )
}
pl_module = M3CoLESModule(
    validation_metric=BatchRecallTopK(
        K=1,
        metric="cosine"
    ),
    head=head,
    seq_encoders=seq_encoders,
    loss=ptls.frames.coles.losses.SoftmaxLoss(),
    optimizer_partial=partial(torch.optim.AdamW,
        lr=0.001,
        weight_decay=1e-4
    ),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR,
        step_size=1,
        gamma=0.9
    )
)

In [142]:
trainer.fit(pl_module, data_module)

defaultdict(<class 'dict'>, {'client_id': '000e047a31e50ba35f71c81962b4eb0b9a2d6080cf23a14f91447317dd14c788', 'trx': {'trx_event_type': tensor([]), 'trx_event_subtype': tensor([]), 'trx_src_type11': tensor([]), 'trx_src_type12': tensor([]), 'trx_dst_type11': tensor([]), 'trx_dst_type12': tensor([]), 'trx_src_type22': tensor([]), 'trx_src_type32': tensor([]), 'trx_event_time': tensor([])}, 'dial': {'dial_embedding': tensor([]), 'dial_event_time': tensor([])}})


KeyError: 'event_time'

In [ ]:
# from ptls.preprocessing import PysparkDataPreprocessor

# TARGETS_DATA_PATH = '/kaggle/working/targets/'
# preprocessor_target = PysparkDataPreprocessor(
#     col_id="client_id",
#     col_event_time="mon",
#     event_time_transformation="dt_to_timestamp",
#     cols_identity=["target_1", "target_2", "target_3", "target_4"],
# )

# for fold in tqdm(range(2)):
    
#     mm_dataset = spark.read.parquet(os.path.join(MM_DATA_PATH , f'fold={fold}'))
#     targets = spark.read.parquet(os.path.join(TARGETS_DATA_PATH , f'fold={fold}'))
#     targets = preprocessor_target.fit_transform(targets)
    
#     mm_dataset = mm_dataset.join(targets, on='client_id', how='left').drop(*['event_time', 'trans_count', 'diff_trans_date'])
#     mm_dataset.write.parquet(os.path.join(MMT_DATA_PATH, f'fold={fold}'))

# Without mm_dataset

In [12]:
spark.read.parquet(os.path.join(DIAL_DATA_PATH, f'fold={1}'))

DataFrame[client_id: string, event_time: array<bigint>, embedding: array<array<float>>]

In [26]:
spark.stop()

**Get trx embeddings**

In [9]:
trx_train = ParquetDataset(ParquetFiles(os.path.join(TRX_DATA_PATH, f'fold={1}')))

In [10]:
map_processed_trx = MemoryMapDataset(
    data=trx_train,
    i_filters=[
        SeqLenFilter(min_seq_len=32),
        ISeqLenLimit(max_seq_len=4096),
        ToTorch(),
    ]
)

**TrxDataLoader**

In [11]:
trx_train = PtlsDataModule(
    train_data=ColesDataset(map_processed_trx,
        splitter=SampleSlices(
            split_count=5,
            cnt_min=25,
            cnt_max=200
        ),
    ),
    train_num_workers=16,
    train_batch_size=256,
)

**Model**

In [12]:
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule
from functools import partial
import torch
trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={"amount": "log"},
    embeddings={
        'event_type': {"in": 58, "out": 24},
        'event_subtype': {"in": 59, "out": 24},
        'src_type11': {"in": 85, "out": 24},
        'src_type12': {"in": 349, "out": 24},
        'dst_type11': {"in": 84, "out": 24},
        'dst_type12': {"in": 417, "out": 24},
        'src_type22': {"in": 90, "out": 24},
        'src_type32': {"in": 91, "out": 24}
    }
)

seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=256,
    type="gru",
)

model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(
        torch.optim.lr_scheduler.StepLR, step_size=30, gamma=0.9
    ),
)

In [13]:
trainer = pl.Trainer(
    max_epochs=12,
    accelerator="cuda" if torch.cuda.is_available() else "cpu",
    enable_progress_bar=True,
)

In [14]:
trainer.fit(model, trx_train)
print(trainer.logged_metrics)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Training: |          | 0/? [00:00<?, ?it/s]

{'loss': tensor(10.6936), 'seq_len': tensor(89.0432)}


**now use pandas over pyspark**

In [ ]:
# import pandas as pd
# FOLD = 1
# mm_dataset = pd.read_parquet(os.path.join(MM_DATA_PATH, f'fold={FOLD}'))

In [ ]:
# mm_dataset

In [ ]:
# test_emb = mm_dataset['dial_embedding'][1].copy()
# print(np.array(test_emb).shape)
# print(np.vstack(test_emb).shape)

**TRX**

380 - среднее количество элементов

10000 - максимальное

**DIAL**

91 - max

5 - avg

In [ ]:
# import datetime

In [ ]:
# def get_min_max_date(dates_list):
#     min_date = None
#     max_date = None
#     for date_list in mm_dataset['trx_event_time']:
#         if type(date_list) is np.ndarray:
#             max_in_list = date_list.max()
#             min_in_list = date_list.min()
#             if not min_date or min_in_list < min_date:
#                 min_date = min_in_list
#             if not max_date or max_in_list > max_date:
#                 max_date = max_in_list
#     return min_date, max_date

In [ ]:
# trx_min_date, trx_max_date = get_min_max_date(mm_dataset['trx_event_time'])
# print(f"""min_date = {datetime.datetime.fromtimestamp(trx_min_date)}
# max_date = {datetime.datetime.fromtimestamp(trx_max_date)}""")
# print((datetime.datetime.fromtimestamp(trx_max_date) - datetime.datetime.fromtimestamp(trx_min_date)).total_seconds() // 3600)

In [ ]:
# trx_max_len = mm_dataset['trx_event_time'].str.len().agg(['max'])['max']
# trx_mean_len = mm_dataset['trx_event_time'].str.len().agg(['mean'])['mean']
# print(f'trx_max_len = {trx_max_len}\ntrx_mean_len = {trx_mean_len}')

In [ ]:
# dial_max_len = mm_dataset['dial_event_time'].str.len().agg(['max'])['max']
# dial_mean_len = mm_dataset['dial_event_time'].str.len().agg(['mean'])['mean']
# print(f'dial_max_len = {dial_max_len}\ndial_mean_len = {dial_mean_len}')

In [ ]:
# trx_date_cols = [col for col in mm_dataset.columns if col.startswith("trx")]
# dial_date_cols = [col for col in mm_dataset.columns if col.startswith("dial")]

In [ ]:
# from tqdm import tqdm

# tqdm.pandas(desc='My bar!')


# def add_null(
#     timestamps_list,
#     max_date,
#     *columns,
    
# ):
#     if type(timestamps_list) is np.ndarray:
#         prev_time = datetime.datetime.fromtimestamp(timestamps_list[0])
#         filled_event_time = []
#         for time in timestamps_list:
#             time = datetime.datetime.fromtimestamp(time)
#             difference_minutes = int((time - prev_time).total_seconds() // 3600)
#             if prev_time < time and difference_minutes > 1:
#                 for minute in range(difference_minutes):
#                     prev_time += datetime.timedelta(hours=1)
#                     filled_event_time.append(int(prev_time.timestamp()))
#             filled_event_time.append(int(time.timestamp()))
    
#         event_time_index = 0
#         new_columns = [[] for i in range(len(columns))]
#         for i in range(len(filled_event_time)):
#             if i < len(filled_event_time) - 1 and filled_event_time[i] == timestamps_list[event_time_index]:
#                 for j in range(len(columns)):
#                     new_columns[j].append(columns[j][event_time_index])
#                 event_time_index += 1
#             else:
#                 for j in range(len(columns)):
#                     new_columns[j].append(0)
                    
#         return [filled_event_time] + list(new_columns)
#     else:
#         return [timestamps_list] + list(columns)
# cols = [col for col in mm_dataset.columns if col.startswith("trx") and col != 'trx_event_time']
# cols_with_time = [col for col in mm_dataset.columns if col.startswith("trx")]
# # add_null(mm_dataset['trx_event_time'][0], *(mm_dataset[col] for col in cols), max_date=trx_max_date)
# result = mm_dataset.progress_apply(
#     lambda row: add_null(row["trx_event_time"], trx_max_date, *(row[col] for col in cols)),
#     axis=1
# )
# result